<a href="https://colab.research.google.com/github/kili-technology/kili-python-sdk/blob/main/recipes/llm_project_setup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# How to Set Up a Kili Project with an LLM Model and Create a Conversation

In this tutorial, we will learn how to set up a Kili project that uses a Large Language Model (LLM), associate an LLM model with the project, and create a conversation using the Kili Python SDK.

Here are the steps we will follow:

1. Creating a Kili project with a custom interface
2. Creating an LLM model
3. Associating the model with the project
4. Creating a conversation

## Creating a Kili Project with a Custom Interface

We will create a Kili project with a custom interface that includes a comparison job and a classification job. This interface will be used for labeling and comparing LLM outputs.

Here's the JSON interface we will use:

In [ ]:
interface = {
    "jobs": {
        "COMPARISON_JOB": {
            "content": {
                "options": {
                    "IS_MUCH_BETTER": {"children": [], "name": "Is much better", "id": "option1"},
                    "IS_BETTER": {"children": [], "name": "Is better", "id": "option2"},
                    "IS_SLIGHTLY_BETTER": {
                        "children": [],
                        "name": "Is slightly better",
                        "id": "option3",
                    },
                    "TIE": {"children": [], "name": "Tie", "id": "option4", "mutual": True},
                },
                "input": "radio",
            },
            "instruction": "Pick the best answer",
            "mlTask": "COMPARISON",
            "required": 1,
            "isChild": False,
            "isNew": False,
        },
        "CLASSIFICATION_JOB": {
            "content": {
                "categories": {
                    "BOTH_ARE_GOOD": {"children": [], "name": "Both are good", "id": "category1"},
                    "BOTH_ARE_BAD": {"children": [], "name": "Both are bad", "id": "category2"},
                },
                "input": "radio",
            },
            "instruction": "Overall quality",
            "mlTask": "CLASSIFICATION",
            "required": 0,
            "isChild": False,
            "isNew": False,
        },
    }
}

Now, we create the project using the `create_project` method:

In [ ]:
from kili.client import Kili

kili = Kili(
    # api_endpoint="https://cloud.kili-technology.com/api/label/v2/graphql",
)
project = kili.create_project(
    title="[Kili SDK Notebook]: LLM Project",
    description="Project Description",
    input_type="LLM_INSTR_FOLLOWING",
    json_interface=interface,
)

project_id = project["id"]
print(f"Project created with ID: {project_id}")

## Creating an LLM Model

We will now create an LLM model in Kili. For this, we need to specify the model's credentials, name, and type.

**Note**: Replace `'YOUR_MODEL_API_KEY'` and `'YOUR_MODEL_ENDPOINT'` with your model's actual credentials.

In [ ]:
model_response = kili.llm.create_model(
    organization_id="your_organization_id",  # Replace with your organization ID
    model={
        "credentials": {"api_key": "YOUR_MODEL_API_KEY", "endpoint": "YOUR_MODEL_ENDPOINT"},
        "name": "Your Model Name",
        "type": "OPEN_AI_SDK",  # Or the appropriate model type
    },
)

model_id = model_response["id"]
print(f"Model created with ID: {model_id}")

## Associating the Model with the Project

Next, we will associate the created model with our project by creating project models with different configurations.

We will create two project models with different temperature settings.

In [ ]:
# First project model with a fixed temperature
first_project_model = kili.llm.create_project_model(
    project_id=project_id,
    model_id=model_id,
    configuration={
        "model": "Your Model Name",
        "temperature": 0.5,
    },
)

first_project_model_id = first_project_model["id"]
print(f"First project model created with ID: {first_project_model_id}")

# Second project model with a temperature range
second_project_model = kili.llm.create_project_model(
    project_id=project_id,
    model_id=model_id,
    configuration={
        "model": "Your Model Name",
        "temperature": {"min": 0.2, "max": 0.8},
    },
)

second_project_model_id = second_project_model["id"]
print(f"Second project model created with ID: {second_project_model_id}")

## Creating a Conversation

Now, we will create a conversation in the project using a prompt.

In [ ]:
conversation = kili.llm.create_conversation(project_id=project_id, prompt="Hello, world!")

print("Conversation created.")

## Summary

In this tutorial, we've successfully set up a Kili project with a custom interface, created an LLM model, associated the model with the project by creating project models, and created a conversation using the Kili Python SDK.